In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta import DeltaTable

In [0]:
import sys
sys.path.append("/Workspace/Users/mayur10594@gmail.com/ETL_project")

In [0]:
import json, yaml, os
from config.utils import load_config
from schemas import user_schema,trans_schema,cards_schema

In [0]:
env='dev'
path="/Workspace/Users/mayur10594@gmail.com/ETL_project/config/etl_main.yaml"
env_config,tables_config=load_config(path,env)

In [0]:
in_path=env_config['incoming_path']

In [0]:
df_trans=spark.read.csv(f"{in_path}/transactions_data.csv",schema=trans_schema,header=True)
df_users=spark.read.csv(f"{in_path}/users_data.csv",schema=user_schema,header=True)
df_cards=spark.read.csv(f"{in_path}/cards_data.csv",schema=cards_schema,header=True)

In [0]:
df_cards=df_cards.withColumn('create_date',current_date()).withColumn('create_user',lit('cpprod'))
df_trans=df_trans.withColumn('create_date',current_date()).withColumn('create_user',lit('cpprod'))
df_users=df_users.withColumn('create_date',current_date()).withColumn('create_user',lit('cpprod'))

In [0]:
df_users.write.format("delta").options(mergeSchema=True).mode('overwrite').saveAsTable('dbfs.users')

In [0]:
%sql update dbfs.users set active_status='A';

In [0]:
df_source=df_users.filter(col('id') <= 10).withColumn('yearly_income',lit(100000)).withColumn('total_debt',lit(100000)).orderBy(col('id'))

In [0]:
df_source=df_source.withColumn('total_debt',col('total_debt').cast('double')).withColumn('yearly_income',col('yearly_income').cast('double'))

In [0]:
df_source2=df_users.filter(col('id').between(11, 20))

In [0]:
df_source2=df_source2.withColumn('id',when(col('id')==20,2222).otherwise(col('id')))

In [0]:
df_source=df_source.union(df_source2)

In [0]:
targetDelta=DeltaTable.forName(spark,'dbfs.users')
df_target=targetDelta.toDF()

In [0]:
from config.utils import run_scd_type2

In [0]:
run_scd_type2(
    spark,
    df_source=df_source,
    target_table_path="dbfs.users",
    config_path="/Workspace/Users/mayur10594@gmail.com/ETL_project/config/scd_config.yaml"
)

In [0]:
%sql select * from dbfs.users where id <=21 or id=2222 order by id;